In [0]:
# Create smaller dataset for Dogs vs. Cats
import os, shutil

original_dataset_dir = 'dogs_vs_cats/train/'
base_dir = 'dogs_vs_cats/data/data_sub'
if not os.path.exists(base_dir):
   os.mkdir(base_dir)

# Create directories
train_dir = os.path.join(base_dir,'train')
if not os.path.exists(train_dir):
   os.mkdir(train_dir)
validation_dir = os.path.join(base_dir,'validation')
if not os.path.exists(validation_dir):
   os.mkdir(validation_dir)
test_dir = os.path.join(base_dir,'test')
if not os.path.exists(test_dir):
   os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir,'cats')
if not os.path.exists(train_cats_dir):
   os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir,'dogs')
if not os.path.exists(train_dogs_dir):
   os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir,'cats')
if not os.path.exists(validation_cats_dir):
   os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
if not os.path.exists(validation_dogs_dir):
   os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
if not os.path.exists(test_cats_dir):
   os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
if not os.path.exists(test_dogs_dir):
   os.mkdir(test_dogs_dir)

# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(5000)]
for fname in fnames:
   src = os.path.join(original_dataset_dir, fname)
   dst = os.path.join(train_cats_dir, fname)
   shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(5000, 5500)]
for fname in fnames:
   src = os.path.join(original_dataset_dir, fname)
   dst = os.path.join(validation_cats_dir, fname)
   shutil.copyfile(src, dst)

# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(5500,6000)]
for fname in fnames:
   src = os.path.join(original_dataset_dir, fname)
   dst = os.path.join(test_cats_dir, fname)
   shutil.copyfile(src, dst)

# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(5000)]
for fname in fnames:
   src = os.path.join(original_dataset_dir, fname)
   dst = os.path.join(train_dogs_dir, fname)
   shutil.copyfile(src, dst)

# Copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(5000,5500)]
for fname in fnames:
   src = os.path.join(original_dataset_dir, fname)
   dst = os.path.join(validation_dogs_dir, fname)
   shutil.copyfile(src, dst)

# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(5500,6000)]
for fname in fnames:
   src = os.path.join(original_dataset_dir, fname)
   dst = os.path.join(test_dogs_dir, fname)
   shutil.copyfile(src, dst)

# Sanity checks
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 5000
total training dog images: 5000
total validation cat images: 500
total validation dog images: 500
total test cat images: 500
total test dog images: 500


In [0]:
# Extract features
import os, shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
img_width = 224
img_height = 224
batch_size = 32

# Instantiate convolutional base
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

def extract_features(directory, sample_count):
   features = np.zeros(shape=(sample_count, 7, 7, 512))  # Must be equal to the output of the convolutional base
   labels = np.zeros(shape=(sample_count))
   # Preprocess data
   generator = datagen.flow_from_directory(directory,
                                           target_size=(img_width,img_height),
                                           batch_size = batch_size,
                                           class_mode='binary')
   # Pass data through convolutional base
   i = 0
   for inputs_batch, labels_batch in generator:
       features_batch = conv_base.predict(inputs_batch)
       features[i * batch_size: (i + 1) * batch_size] = features_batch
       labels[i * batch_size: (i + 1) * batch_size] = labels_batch
       i += 1
       if i * batch_size >= sample_count:
           break
   return features, labels

train_size = 10000
validation_size = 1000
test_size = 1000

dataset_home = 'dogs_vs_cats/data/data_sub/'

train_dir = dataset_home + "train"
validation_dir = dataset_home + "validation"
test_dir = dataset_home + "test"

In [0]:
train_features, train_labels = extract_features(train_dir, train_size)  # Agree with our small dataset size
validation_features, validation_labels = extract_features(validation_dir, validation_size)
test_features, test_labels = extract_features(test_dir, test_size)

Found 10000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
# Define model
from keras import models
from keras import layers
from keras import optimizers
epochs = 25
model = models.Sequential()
model.add(layers.Flatten(input_shape=(7,7,512)))
model.add(layers.Dense(256, activation='relu', input_dim=(7*7*512)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()
# Compile model
model.compile(optimizer=optimizers.Adam(),
             loss='binary_crossentropy',
             metrics=['acc'])
model.fit(train_features, train_labels,
                   epochs=epochs,
                   batch_size=batch_size,
                   validation_data=(validation_features, validation_labels))


W0825 06:29:52.157272 140160046565248 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0825 06:29:52.193544 140160046565248 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0825 06:29:52.200757 140160046565248 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 6,423,041
Trainable params: 6,423,041
Non-trainable params: 0
_________________________________________________________________
Train on 10000 samples, validate on 1000 samples
Epoch 1/25
10000/10000 [==============================] - 6s 576us/step - loss: 0.3466 - acc: 0.8626 - val_loss: 0.2029 - val_acc: 0.9170
Epoch 2/25
10000/10000 [==============================] - 4s 407us/step - los

In [0]:
# Concatenate training and validation sets
svm_features = np.concatenate((train_features, validation_features))
svm_labels = np.concatenate((train_labels, validation_labels))

In [0]:
# Build model

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

X_train, y_train = svm_features.reshape(300,7*7*512), svm_labels

param = [{
         "C": [0.01, 0.1, 1, 10, 100]
        }]
 
svm = LinearSVC(penalty='l2', loss='squared_hinge')  # As in Tang (2013)
clf = GridSearchCV(svm, param, cv=10)
clf.fit(X_train, y_train)

ValueError: ignored

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

print("\nAccuracy score (mean):")
print(np.mean(cross_val_score(clf, X_train, y_train, cv=10)))
print("\nAccuracy score (standard deviation):")


Accuracy score (mean):


NameError: ignored